In [1]:
#Analysis and data-loading functions
"""
Objective:
    *Docs
        - Create summary to easily access description for different data-attributes
    *Loading
        - Create functions that load arbitrairy amount of json-data for the same
        type of statistics.
        - Create functions to concatinate json-data of different type
    *Analysis
        - Visualize data-attributes in graphs
"""

'\nObjective:\n    *Docs\n        - Create summary to easily access description for different data-attributes\n    *Loading\n        - Create functions that load arbitrairy amount of json-data for the same\n        type of statistics.\n        - Create functions to concatinate json-data of different type\n    *Analysis\n        - Visualize data-attributes in graphs\n'

In [2]:
from directory import Directory
import pandas as pd
from pprint import pprint
from functools import reduce 
import mysql.connector
dirs = Directory()

In [30]:
dirs = Directory()

"""
Updates .json files with new data from the api if the json exists

Looks if fixtureID exists in .json, if it doesn't exist it gets append
"""

def deep_get(dictionary, keys, default=None):
    """Get values of nested keys from dict
        Args:
            dictionary(dict): Dict with nested keys
            keys(dict.keys()): "." separated chain of nested keys, ex "info.player.name"
    """
    return reduce(lambda d, key: d.get(key, default) if isinstance(d, dict) else default, keys.split("."), dictionary)

def load_player_stats(year):
    """Load player_stats json files into a container
        Args:
            year(int): year of player_stats json
    """
    file = f'EN_PR_{start_year}_playerstats.json'
    stats_file = dirs.load_json(file, '..', 'json', 'params', 'stats')
    stats_file.append({'season': year})
    return stats_file

"""Read stats from ...playerstats.json into Dataframe,
can later be used to import data into DataBase
"""

def read_playerstats(data):
    """Read stats from ...playerstats.json into flattened
    list of dicts. 
    """
    stats_all = []
    stats_temp = {}
    for d in data:
        stats_temp = {}
        if 'stats' in d:
            stats = d['stats']
            for dicts in stats:
                stats_temp['id'] = dicts.get('id')
                if dicts.get('name') != None:
                    stats_temp[dicts.get('name')] = dicts.get('value')
            stats_all.append(stats_temp)
    return stats_all

def read_playerinfo(data):
    """Read info from ...playerstats.json into flattened
    list of dicts. 
    """
    info_all = []
    for d in data:
        stats_temp = {}
        if 'info' in d:
            stats = d['info']
            stats_temp['age'] = stats.get('age')
            stats_temp['id'] = stats.get('id')
            stats_temp['birth'] = deep_get(stats, 'birth.date.label')
            stats_temp['birth_exact'] = deep_get(stats, 'birth.date.millis')
            stats_temp['country'] = deep_get(stats, 'birth.country.country')
            stats_temp['isoCode'] = deep_get(stats, 'birth.country.isoCode')
            stats_temp['loan'] = deep_get(stats, 'info.loan')
            stats_temp['position'] = deep_get(stats, 'info.position')
            stats_temp['positionInfo'] = deep_get(stats, 'info.positionInfo')
            stats_temp['shirtNum'] = deep_get(stats, 'info.shirtNum')
            stats_temp['name'] = deep_get(stats, 'name.display')
            stats_temp['first'] = deep_get(stats, 'name.first')
            stats_temp['last'] = deep_get(stats, 'name.last')
            stats_temp['nationalTeam'] = deep_get(stats, 'nationalTeam.country')
            stats_temp['playerId'] = stats.get('playerId')
        stats_temp['season'] = data[-1].get('season')
        info_all.append(stats_temp)
    return info_all
               
            
start_year = 2017
end_year = 2020
df = pd.DataFrame()
data = []
for i in range(start_year, end_year):
    data_sample = load_player_stats(i)
    stats = read_playerstats(data_sample)
    info = read_playerinfo(data_sample)
    df1 = pd.DataFrame(stats)
    df2 = pd.DataFrame(info)
    df3 = pd.merge(df1, df2, on='id')
    df = df.append(df3)
df.to_csv('player_stats.csv', encoding='utf-8')

    


2019
2017


In [1]:
def func(*args):
    string = "/".join((*args))
    print(string)

In [2]:
func('ar', 'be', 'te')

TypeError: join() takes exactly one argument (3 given)

In [ ]:
import sqlite3
from sqlite3 import Error

def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

connection = create_connection("test_db.sqlite")


In [2]:
str_j = 'print'
str_j('hello')

TypeError: 'str' object is not callable